# Length

In [848]:
import os
import pandas
import numpy as np
import pandas as pd
import re
import ast
import json
from CS_feature_extractor import *
DATA_DIR = '../data'

# Experiment Config
DF_NAME = 'MathQA'

NUM_OF_SAMPLES = 500
NUM_OF_COT = 40
MODEL = 'gpt-3.5-turbo-0125'
storage_dir = os.path.join(DATA_DIR, f'Evaluation_CoTs/Algo_Design_Data')
hard_file_path = os.path.join(storage_dir, f'{DF_NAME}_hard.json')
easy_file_path = os.path.join(storage_dir, f'{DF_NAME}_easy.json')

In [849]:
def sigmoid(x):
    return 1 / (1 + np.exp(-(x-5)))

In [850]:
with open(hard_file_path,'r') as f:
    data = json.load(f)
hard = pd.DataFrame(data)
hard['size_of_cot'] = [[(i+1)^2 for i in range(40)] for j in range(500)]
for row_idx in range(len(hard)):
    hard['SIM_COT_AGG'][row_idx][0] = 0.5
# hard['SIM_COT_AGG'] = hard['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,c,"[54, c, 111, c, 111, a, b, a, d, 11, c, 112, 1...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[12, 6, 4, 5, 6, 3, 0, 4, 6, 12, 6, 4, 6, 3, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5518672199170125, 0.6051282051282052, ...","[0.5, 0.5495867768595042, 0.5443548387096775, ...","[0.0, 0.5518672199170125, 0.5955873583780561, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,c,"[a, a, a, a, a, c, a, a, a, a, a, a, a, a, a, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 5, 5, 5, 3, 5, 5, 6, 3, 4, 3, 8, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 5, 0, 0, 0, 5, 2, 0, 2, 0, 5, 9, 0, ...",Challenging Math,"[0.0, 0.6423357664233577, 0.5263157894736843, ...","[0.5, 0.6423357664233577, 0.5428571428571429, ...","[0.0, 0.6423357664233577, 0.5515014530190507, ...","[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,a,"[a, a, a, a, a, error, error, a, a, a, error, ...","[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 2, 2, 2, 10, 10, 2, 2, 0, 10, 0, 0, 9, ...",Challenging Math,"[0.0, 0.943089430894309, 0.943089430894309, 0....","[0.5, 0.943089430894309, 0.967479674796748, 0....","[0.0, 0.943089430894309, 0.9715447154471545, 0...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ...","[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,a,"[error, a, a, a, a, a, a, a, a, a, 100%, 100%,...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 6, 5, 6, 5, 4, 3, 4, 4, 4, 7, 6, 3, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 2, 9, 2, 9, 9, 2, 0, 9, 2, 2, 0, 0, 5, 0,...",Challenging Math,"[0.0, 0.012422360248447228, 0.5151515151515151...","[0.5, 0.012345679012345734, 0.5276381909547738...","[0.0, 0.012422360248447228, 0.2717601547388781...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,b,"[b, 36, b, b, b, b, e, b, b, b, e, b, b, b, b,...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 6, 3, 3, 3, 3, 4, 3, 3, 3, 5, 5, 3, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5091575091575091, 0.49416342412451364,...","[0.5, 0.5072992700729927, 0.5226480836236933, ...","[0.0, 0.5091575091575091, 0.5772703913075399, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,d,"[d, d, d, d, d, d, d, d, d, d, d, d, d, d, d, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 6, 1, 6, 7, 6, 5, 6, 6, 5, 6, 6, 7, 4, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...",Challenging Math,"[0.0, 0.8583690987124464, 0.31759656652360513,...","[0.5, 0.8583690987124464, 0.3157894736842105, ...","[0.0, 0.8583690987124464, 0.32119144565496494,...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",

In [851]:
with open(easy_file_path,'r') as f:
    data = json.load(f)
easy = pd.DataFrame(data)
easy['size_of_cot'] = [[i+1 for i in range(40)] for j in range(500)]
for row_idx in range(len(easy)):
    easy['SIM_COT_AGG'][row_idx][0] = 0.5
easy['SIM_COT_AGG'] = easy['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
easy

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,c,"[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.9591836734693877, 0.9333333333333333, ...","[0.0033464254621424277, 0.017135510842262905, ...","[0.0, 0.9591836734693877, 0.9529680365296804, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,b,"[e, e, c, a, e, 9-May, c, d, c, c, e, 9-May, c...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 4, 3, 4, 3, 3, 3, 6, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.8916666666666666, 0.9285714285714286, ...","[0.0033464254621424277, 0.016037703882864972, ...","[0.0, 0.8916666666666666, 0.8857733175914995, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,2,c,"[c, a, c, c, c, d, c, c, c, c, c, c, 7, c, c, ...","[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[6, 6, 9, 6, 6, 4, 5, 6, 6, 7, 6, 6, 6, 6, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.8622754491017964, 0.6601941747572815, ...","[0.0033464254621424277, 0.01541675139607848, 0...","[0.0, 0.8622754491017964, 0.6989089685667595, ...","[0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,3,d,"[d, d, a, 18-Jan, b, b, b, b, b, d, b, 18-Jan,...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[4, 3, 4, 3, 6, 6, 3, 5, 5, 3, 3, 3, 7, 7, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.6637554585152838, 0.7188940092165899, ...","[0.0033464254621424277, 0.01193844504034025, 0...","[0.0, 0.6637554585152838, 0.6420557002604689, ...","[0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,4,a,"[a, b, a, d, b, a, b, cannot be determined, b,...","[1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, ...","[4, 6, 5, 3, 8, 3, 6, 3, 3, 3, 2, 3, 4, 6, 6, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.5139442231075697, 0.5780590717299579, ...","[0.0033464254621424277, 0.009315566912637062, ...","[0.0, 0.5139442231075697, 0.6768800031546985, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,b,"[b, b, b, b, b, b, b, b, 17, b, 17, 17, b, b, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[4, 6, 5, 6, 9, 5, 6, 4, 8, 4, 8, 8, 8, 8, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.6560509554140128, 0.7225806451612904, ...","[0.0033464254621424277, 0.011799870229907203, ...","[0.0, 0.6560509554140128, 0.7371292487551419, ...","[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[1, 2, 3, 4, 5, 6, 

In [852]:
hard = pd.concat([hard,easy])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,c,"[54, c, 111, c, 111, a, b, a, d, 11, c, 112, 1...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[12, 6, 4, 5, 6, 3, 0, 4, 6, 12, 6, 4, 6, 3, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5518672199170125, 0.6051282051282052, ...","[0.5, 0.5495867768595042, 0.5443548387096775, ...","[0.0, 0.5518672199170125, 0.5955873583780561, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,c,"[a, a, a, a, a, c, a, a, a, a, a, a, a, a, a, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 5, 5, 5, 3, 5, 5, 6, 3, 4, 3, 8, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 5, 0, 0, 0, 5, 2, 0, 2, 0, 5, 9, 0, ...",Challenging Math,"[0.0, 0.6423357664233577, 0.5263157894736843, ...","[0.5, 0.6423357664233577, 0.5428571428571429, ...","[0.0, 0.6423357664233577, 0.5515014530190507, ...","[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,a,"[a, a, a, a, a, error, error, a, a, a, error, ...","[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 2, 2, 2, 10, 10, 2, 2, 0, 10, 0, 0, 9, ...",Challenging Math,"[0.0, 0.943089430894309, 0.943089430894309, 0....","[0.5, 0.943089430894309, 0.967479674796748, 0....","[0.0, 0.943089430894309, 0.9715447154471545, 0...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, ...","[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,a,"[error, a, a, a, a, a, a, a, a, a, 100%, 100%,...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 6, 5, 6, 5, 4, 3, 4, 4, 4, 7, 6, 3, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 2, 9, 2, 9, 9, 2, 0, 9, 2, 2, 0, 0, 5, 0,...",Challenging Math,"[0.0, 0.012422360248447228, 0.5151515151515151...","[0.5, 0.012345679012345734, 0.5276381909547738...","[0.0, 0.012422360248447228, 0.2717601547388781...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,b,"[b, 36, b, b, b, b, e, b, b, b, e, b, b, b, b,...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 6, 3, 3, 3, 3, 4, 3, 3, 3, 5, 5, 3, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5091575091575091, 0.49416342412451364,...","[0.5, 0.5072992700729927, 0.5226480836236933, ...","[0.0, 0.5091575091575091, 0.5772703913075399, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,b,"[b, b, b, b, b, b, b, b, 17, b, 17, 17, b, b, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[4, 6, 5, 6, 9, 5, 6, 4, 8, 4, 8, 8, 8, 8, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.6560509554140128, 0.7225806451612904, ...","[0.0033464254621424277, 0.011799870229907203, ...","[0.0, 0.6560509554140128, 0.7371292487551419, ...","[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 1, 1

# Early stopping and SC 

In [853]:
from collections import Counter

def calculate_correctness(df):
    # Define a helper function to determine the majority and compare it with the correct answer
    def check_majority(answers, correct):
        if not answers:
            return 0
        # Count the occurrences of each answer and find the most common one
        most_common = Counter(answers).most_common(1)[0][0]
        # Compare the most common answer with the correct answer
        return 1 if most_common == str(correct) else 0

    # Apply the helper function across the DataFrame rows
    df['SC_correctness'] = df.apply(lambda row: check_majority(row['CoT answers'], row['correct answer']), axis=1)
    return df

hard = calculate_correctness(hard)

In [854]:
hard.SC_correctness.value_counts()

SC_correctness
1    570
0    430
Name: count, dtype: int64

In [855]:
easy = calculate_correctness(easy)
easy.SC_correctness.value_counts()

SC_correctness
1    292
0    208
Name: count, dtype: int64

In [856]:
import pandas as pd

def evaluate_answers_with_window(df, window_size):
    # Helper function to evaluate answers with a sliding window
    def evaluate_window(answers, correct):
        # Track the number of steps (checks) taken
        steps = window_size-1
        
        # If the list is shorter than the window size, adjust the window size to the length of the list
        window_size_adjusted = min(window_size, len(answers))
        
        # Slide through the list with the adjusted window size
        for i in range(len(answers) - window_size_adjusted + 1):
            # Check the window content
            window = answers[i:i + window_size_adjusted]
            steps += 1
            
            # If all elements in the window are the same, evaluate correctness
            if window.count(window[0]) == window_size_adjusted:
                return 1 if window[0] == str(correct) else 0, steps
        
        # If no unanimous window is found, use the same value as SC_correctness and set steps to the length of the list
        majority = Counter(answers).most_common(1)[0][0]
        correctness = 1 if majority == str(correct) else 0
        return correctness, len(answers)

    # Apply the function to each row in the DataFrame
    result = df.apply(lambda row: evaluate_window(row['CoT answers'], row['correct answer']), axis=1)
    df['ES_correctness'] = result.apply(lambda x: x[0])
    df['ES_steps'] = result.apply(lambda x: x[1])

    return df

easy = evaluate_answers_with_window(easy,5)

In [857]:
easy.ES_correctness.value_counts()

ES_correctness
1    297
0    203
Name: count, dtype: int64

In [858]:
easy.ES_steps.mean()

18.648

In [859]:

hard = evaluate_answers_with_window(hard,5)
hard.ES_correctness.value_counts()

ES_correctness
1    573
0    427
Name: count, dtype: int64

In [860]:
hard.ES_steps.mean()

19.301

In [861]:
feature_li = [
 # 'LEN',
 'QUA_IM',
 'DIF_IV',
 # 'DIF_SUB',
 # 'SIM_COT_BIGRAM',
 'SIM_COT_AGG',
 # 'SIM_COT_PW',
 'SIM_AC_BIGRAM',
 'SIM_AC_AGG',
 'SIM_AC_PW',
    'size_of_cot'

 
  ]

# Visualize

In [862]:
import pandas as pd

def concatenate_columns(df,data_columns, outcome_column):
    # Initialize an empty dictionary to store the concatenated data
    concatenated_data = {}
    
    # Get the number of rows based on the length of the outcome column
    num_rows = len(df)
    
    # Iterate over each column
    for column in data_columns+[outcome_column]:
        # Initialize an empty list to store the concatenated values for the current column
        concatenated_values = []
        
        # Iterate over each row
        for i in range(num_rows):
            # Get the list of values for the current column and row
            
            values = df[column].iloc[i]
            
            # Concatenate the values into a single string
            concatenated_values += list(values)
            
        
        # Add the concatenated values to the dictionary with the column name as the key
        concatenated_data[column] = concatenated_values
        print(len(concatenated_values))
    # Add the outcome column to the concatenated data dictionary
    
    # Create a DataFrame from the concatenated data dictionary
    hard_final = pd.DataFrame(concatenated_data)
    
    return hard_final

hard_final = concatenate_columns(hard,feature_li,'Correctness')
easy_final = concatenate_columns(easy,feature_li,'Correctness')

40000
40000
40000
40000
40000
40000
40000
40000
20000
20000
20000
20000
20000
20000
20000
20000


In [863]:
hard_final['cot_answer'] = np.repeat(hard['correct answer'].values,40)

In [864]:
easy_final['cot_answer'] = np.repeat(easy['correct answer'].values,40)

In [865]:
from collections import Counter
print('--------------EASY--------------')
for col in easy_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(easy_final[col]))
print('--------------HARD--------------')
for col in hard_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(hard_final[col]))

--------------EASY--------------
QUA_IM : Counter({0: 19952, 1: 48})
DIF_IV : Counter({0: 18771, 2: 870, 5: 199, 9: 81, 10: 78, 1: 1})
SIM_AC_BIGRAM : Counter({1: 11231, 0: 8769})
SIM_AC_AGG : Counter({1: 12909, 0: 7091})
SIM_AC_PW : Counter({1: 10795, 0: 9205})
size_of_cot : Counter({1: 500, 2: 500, 3: 500, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, 9: 500, 10: 500, 11: 500, 12: 500, 13: 500, 14: 500, 15: 500, 16: 500, 17: 500, 18: 500, 19: 500, 20: 500, 21: 500, 22: 500, 23: 500, 24: 500, 25: 500, 26: 500, 27: 500, 28: 500, 29: 500, 30: 500, 31: 500, 32: 500, 33: 500, 34: 500, 35: 500, 36: 500, 37: 500, 38: 500, 39: 500, 40: 500})
Correctness : Counter({1: 10142, 0: 9858})
cot_answer : Counter({'c': 5400, 'b': 4080, 'a': 3800, 'd': 3600, 'e': 3120})
--------------HARD--------------
QUA_IM : Counter({0: 39888, 1: 112})
DIF_IV : Counter({0: 34722, 2: 2939, 10: 1024, 5: 851, 9: 452, 1: 6, 11: 2, 3: 2, 6: 2})
SIM_AC_BIGRAM : Counter({1: 21811, 0: 18189})
SIM_AC_AGG : Counter({1: 25315, 0: 1

# CS based performance

In [866]:
def log_reg(features):
    coe = [-0.1, -5 , -1  , 3 , 2]
    lincomb = sum([coe[i]*features[i] for i in range(len(coe))]) -1.5
    return 1 / (1 + np.exp(-lincomb))
log_reg([5,0,0,1,1])

0.9525741268224334

In [867]:
df = hard
df_concate = hard_final
threshold = 0.36
warm_up_steps = 0

In [868]:
# df_concate['confidence_score']= cls_model.predict_proba(df_concate[feature_li])[:,1]
df_concate['confidence_score']=df_concate[feature_li].apply(log_reg,axis =1)
lists = [df_concate['confidence_score'].iloc[i:i+40].tolist() for i in range(0, len(df_concate), 40)]
df['confidence_score'] = lists

In [869]:
def normalize_cs(cs_li, threshold):
    cs_arr = np.array(cs_li)
    # normalized_cs = [(cs-threshold)/(1-threshold) if cs > threshold else (cs-threshold)/(threshold) for cs in cs_arr]
    normalized_cs = cs_arr-threshold
    # normalized_cs = [0.5 if cs > threshold else -0.5 for cs in cs_arr]
    return np.array(normalized_cs)

In [873]:
def stop_con1(individual_cs):
    cumulative_difference = (individual_cs[warm_up_steps:]).cumsum()
    stop_idx = np.argmax(cumulative_difference > 0.5)
    return stop_idx
def stop_con2(individual_cs):
    buffer = []
    for idx,cs in enumerate(individual_cs):
        if cs>0:
            buffer.append(idx)
        if len(buffer) == 5:
            return buffer[-1]

In [874]:
CS_Answer = []
CS_correctness = []
CS_steps = []
for row_idx in range(len(df)): 
    test_row = df.iloc[row_idx]
    individual_cs = normalize_cs(test_row['confidence_score'],threshold)
    # individual_cs = test_row['confidence_score'][warm_up_steps:] - threshold
    stop_idx = stop_con2(individual_cs)
    
    if stop_idx:
        num_of_steps = warm_up_steps+stop_idx
    else: num_of_steps = 40
    answers = test_row['CoT answers'][:num_of_steps]
    scores = individual_cs[:num_of_steps]
    print(scores)
    weighted_votes = Counter()
    for answer, score in zip(answers, scores):
        if score >0:
            weighted_votes[answer] += score
    # Find the answer with the highest total score
    if (len(weighted_votes)) == 0:
        for answer, score in zip(answers, scores):
            weighted_votes[answer] += score
    print(weighted_votes)
    result = max(weighted_votes, key=weighted_votes.get)
    CS_Answer.append(result)
    CS_correctness.append(1 if result == str(test_row['correct answer']) else 0)
    CS_steps.append(num_of_steps)
df['CS_Answer'] = CS_Answer
df['CS_correctness'] = CS_correctness
df['CS_steps'] = CS_steps
df

[-0.24079708 -0.24590586 -0.24537596 -0.24368286 -0.25326231 -0.23640101
 -0.23829168 -0.227364   -0.24246943 -0.22636925  0.14304874 -0.23193555
 -0.2361204  -0.22382531  0.15742617 -0.23907286  0.14408003 -0.24064571
  0.1784685 ]
Counter({'c': 0.6230234400068357})
[-0.24079708  0.58571351  0.5906     -0.36        0.58734143 -0.25499941
  0.12913215 -0.36       -0.35920954]
Counter({'a': 1.8927870771397448})
[-0.24079708 -0.35941487 -0.35942896 -0.35941566 -0.35924521 -0.36
 -0.36       -0.35996509 -0.35930384  0.5791852  -0.36        0.08414436
  0.57946212 -0.36       -0.36       -0.25019979 -0.36       -0.36
 -0.36       -0.36        0.11039549]
Counter({'a': 1.3531871717165123})
[-0.36       -0.35998999 -0.36       -0.35928504 -0.36       -0.36
 -0.35915225  0.58944906 -0.36       -0.35919112 -0.35999488  0.3459995
  0.1472309  -0.36       -0.25438762 -0.36       -0.36        0.11728837
 -0.36       -0.36       -0.36       -0.35995803 -0.36       -0.36
 -0.36      ]
Counter({'a':

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,...,SIM_AC_AGG,SIM_AC_PW,size_of_cot,SC_correctness,ES_correctness,ES_steps,confidence_score,CS_Answer,CS_correctness,CS_steps
0,0,c,"[54, c, 111, c, 111, a, b, a, d, 11, c, 112, 1...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[12, 6, 4, 5, 6, 3, 0, 4, 6, 12, 6, 4, 6, 3, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5518672199170125, 0.6051282051282052, ...","[0.5, 0.5495867768595042, 0.5443548387096775, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,40,"[0.11920292202211755, 0.11409414183828431, 0.1...",c,1,19
1,1,c,"[a, a, a, a, a, c, a, a, a, a, a, a, a, a, a, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 5, 5, 5, 3, 5, 5, 6, 3, 4, 3, 8, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 5, 0, 0, 0, 5, 2, 0, 2, 0, 5, 9, 0, ...",Challenging Math,"[0.0, 0.6423357664233577, 0.5263157894736843, ...","[0.5, 0.6423357664233577, 0.5428571428571429, ...",...,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,5,"[0.11920292202211755, 0.9457135072779629, 0.95...",a,0,9
2,2,a,"[a, a, a, a, a, error, error, a, a, a, error, ...","[1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 2, 2, 2, 10, 10, 2, 2, 0, 10, 0, 0, 9, ...",Challenging Math,"[0.0, 0.943089430894309, 0.943089430894309, 0....","[0.5, 0.943089430894309, 0.967479674796748, 0....",...,"[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,5,"[0.11920292202211755, 0.0005851310415358816, 0...",a,1,21
3,3,a,"[error, a, a, a, a, a, a, a, a, a, 100%, 100%,...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 6, 5, 6, 5, 4, 3, 4, 4, 4, 7, 6, 3, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 2, 9, 2, 9, 9, 2, 0, 9, 2, 2, 0, 0, 5, 0,...",Challenging Math,"[0.0, 0.012422360248447228, 0.5151515151515151...","[0.5, 0.012345679012345734, 0.5276381909547738...",...,"[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,6,"[2.6102790696677047e-23, 1.0005699425639777e-0...",a,1,25
4,4,b,"[b, 36, b, b, b, b, e, b, b, b, e, b, b, b, b,...","[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 6, 3, 3, 3, 3, 4, 3, 3, 3, 5, 5, 3, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Challenging Math,"[0.0, 0.5091575091575091, 0.49416342412451364,...","[0.5, 0.5072992700729927, 0.5226480836236933, ...",...,"[0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,21,"[0.11920292202211755, 0.11843867314042894, 0.4...",b,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,b,"[b, b, b, b, b, b, b, b, 17, b, 17, 17, b, b, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[4, 6, 5, 6, 9, 5, 6, 4, 8, 4, 8, 8, 8, 8, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Math,"[0.0, 0.6560509554140128, 0.7225806451612904, ...","[0.0033464254621424277, 0.011799870229907203, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1,5,"[0.1819269466443896, 0.9703501560544446, 0.969...",b,1,5
496,496,e,"[e, e, e, e, e,

In [875]:
print(df.ES_steps.mean(),df.CS_steps.mean())
print()
print(df.ES_correctness.value_counts(),df.CS_correctness.value_counts())
print()
print(df.ES_correctness.sum()/len(df),df.CS_correctness.sum()/len(df))

19.301 10.301

ES_correctness
1    573
0    427
Name: count, dtype: int64 CS_correctness
1    564
0    436
Name: count, dtype: int64

0.573 0.564


In [754]:

df_t07 = pd.read_json('../data/T0.7.jsonl', lines=True)

In [522]:
df_t07['generated_answer'].iloc[0][3]

"A: Janet's ducks lay 16 eggs per day. She eats 3 and uses 4 for muffins every day. So she has 16 - 3 - 4 = 9 eggs left to sell. She can sell the 9 eggs for 9 x 2 = $18 per day. The answer is $18."

In [1414]:
import re
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"


def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS
extract_answer(df_t07.iloc[0]['answer'])

'18'

In [1343]:
tem_1 = df_t07['generated_answer'].iloc[0][3]
number_list = re.findall(r"\d+\.?\d*", tem_1)
try:
    predict1 = number_list[-1].strip('.')
except:
    predict1 = -1000
predict1

'18'

In [937]:
number_list

['16', '3', '4', '16', '3', '4', '9', '9', '9', '2', '18', '18.']